# Model complexity, context specification and assembly policies
In this demo we explore the effects of specified conditions on Agents (e.g. bound conditions, modification conditions)  and assembly policies on the combinatorial complexity of dynamical models.

First, we import INDRA's TRIPS input API and PySB model assembler.

In [ ]:
from indra import trips
from indra.assemblers import PysbAssembler

# Below is some bookkeeping code needed to display reaction network graphs
%matplotlib inline
from pysb.tools import render_reactions
import pygraphviz, subprocess
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
def draw_reaction_network(model):
    pygraphviz.AGraph(render_reactions.run(model)).draw('model.dot', prog='dot')
    subprocess.call('dot -T png -o model.png model.dot'.split(' '))
    img = mpimg.imread('model.png')
    plt.figure(figsize=(50, 50))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

## Model1: RAF to ERK without specifying agent context
In the first case, two binding events and a phosphorylation is described with no additional context specified on any of the proteins.

In [ ]:
tp = trips.process_text('RAF binds Vemurafenib, RAF phosphorylates MEK and MEK phosphorylates ERK.')

This yields 3 INDRA Statements, as follows. Here empty parentheses after the Agent names indicate that there is no additional context specified on them.

In [ ]:
tp.statements

### Assembly with one-step policy
We now assemble this model using the default `one_step` policy and store it in the `model1_one` variable.

In [ ]:
pa = PysbAssembler()
pa.add_statements(tp.statements)
pa.make_model(policies='one_step')

In [ ]:
model1_one = pa.model

The model has 4 Monomers and 4 Rules.

In [ ]:
model1_one.monomers

In [ ]:
model1_one.rules

Let's examine the last rule which corresponds to MEK phosphorylating ERK. Here, `MEK()` appears without additional context specified. This means that the rule will apply to **any** form of `MEK`, for instance, MEK that is unphosphorylated. 

We now generate the rule-based model into a reaction network form using PySB's interface to BioNetGen.

In [ ]:
from pysb.bng import generate_equations
generate_equations(model1_one)

We can now plot the reaction network to examine the model. Each colored node of the reaction network is a molecular species, reactions are represented by gray nodes, and arrows show species being consumed and produced by each reaction.

In [ ]:
draw_reaction_network(model1_one)

We see from the reaction network that RAF is able to phosphorylate MEK whether or not it is bound to Vemurafenib, and MEK phosphorylates ERK whether or not it is phosphorylated. 

### Assembly with two-step policy
Let's now assemble the same model with the `two-step` policy. This will result in a model detailed model in which MEK first binds ERK reversibly, and phosphorylated ERK is released from the MEK-ERK complex. We will store this model in the `model1_two` variable.

In [ ]:
pa.make_model(policies='two_step')

In [ ]:
model1_two = pa.model

In [ ]:
model1_two.monomers

In [ ]:
for rule in model1_two.rules:
    print(rule.rule_expression)

We can now generate the reaction network for `model1_two` and inspect the reaction network that is created.

In [ ]:
generate_equations(model1_two)

In [ ]:
model1_two.species

In [ ]:
draw_reaction_network(model1_two)

The two-step policy produced a total of 13 molecular species and 19 reactions. ERK now appears in 6 possible forms:
- ERK(phospho='u', map2k=None)
- ERK(phospho='p', map2k=None)
- ERK(phospho='u', map2k=1) % MEK(phospho='u', map3k=None, mapk=1)
- ERK(phospho='u', map2k=1) % MEK(phospho='p', map3k=None, mapk=1)
- ERK(phospho='u', map2k=1) % MEK(phospho='u', map3k=2, mapk=1) % RAF(vemurafenib=None, map2k=2)
- ERK(phospho='u', map2k=1) % MEK(phospho='u', map3k=2, mapk=1) % RAF(vemurafenib=3, map2k=2) % VEMURAFENIB(map3k=3)

This means that our initial description allowed for the possibility of ERK, MEK, RAF and Vemurafenib all simultaneously being in a complex. While the existence of such a complex is not impossible, we can introduce additional assumptions to simplify the model.

## Model2: RAF to ERK with specifying context
In this model we introduce additional assumptions (by explicitly making them part of the model definition) to the previous model. In particular, we add additional context on the agents to make causal structure explicit and simplify the model.

### Assembly with two-step policy

In [ ]:
tp = trips.process_text('RAF binds Vemurafenib. '
                        'RAF not bound to Vemurafenib phosphorylates MEK. '
                        'Phosphorylated MEK not bound to RAF phosphorylates ERK.')

The INDRA Statements extracted by processing this text are shown below.

In [ ]:
tp.statements

We see that some agents are now subject to additional conditions, for instance, `RAF(bound: [VEMURAFENIB, False])` specifies that RAF should not be bound to Vemurafenib in order to phosphorylate MEK.

Let's not assemble the model and generate the reaction network.

In [ ]:
pa = PysbAssembler()
pa.add_statements(tp.statements)
pa.make_model(policies='two_step')

In [ ]:
generate_equations(pa.model)

In [ ]:
draw_reaction_network(pa.model)

The model is now significantly simpler with a total of 7 reactions down from 19 in the previous model.

### Assembly with Michaelis-Menten policy

As an alternative to the two-step policy, we can assemble the same model using a simpler, Michaelis-Menten policy. In this case, both phosphorylation processes are effectively modeled as one-step but their kinetic rates will account for enzyme saturation.

In [ ]:
pa.make_model(policies='michaelis_menten')

In [ ]:
generate_equations(pa.model)

In [ ]:
draw_reaction_network(pa.model)

As we see, this model contains 7 individual species with 3 reactions in total.